# Importing the libraries

In [69]:
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
from shapely.geometry import mapping
import pyproj
import pandas as pd
import harmonize as hz

# Importing the dataset

In [70]:
# Create a path to the data directory
path_data = "../data/Raw/"

In [76]:
# Open the dataset list_xy and list_xdimydim
list_xy = xr.open_dataset( path_data + "list_xy.nc", decode_coords='all')
list_xdimydim = xr.open_dataset(path_data + "list_xdimydim.nc", decode_coords='all')
first_cube = xr.open_dataset(path_data + "data_merged.nc", decode_coords='all')


In [77]:
first_cube

<xarray.Dataset>
Dimensions:            (x: 298, y: 253, time: 4353, ydim: 298, xdim: 360)
Coordinates:
  * x                  (x) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
  * y                  (y) float64 5.018e+06 5.017e+06 ... 4.786e+06 4.785e+06
    crs                int32 ...
  * time               (time) object 2010-02-01 00:00:00 ... 2022-01-01 00:00:00
  * ydim               (ydim) float64 4.785e+06 4.786e+06 ... 5.018e+06
  * xdim               (xdim) float64 3.415e+05 3.424e+05 ... 6.167e+05
    band               int32 ...
    spatial_ref        int32 ...
Data variables:
    ET_500m            (time, y, x) float64 ...
    FireMask           (time, ydim, xdim) float64 ...
    First_Day          (time, ydim, xdim) float64 ...
    Last_Day           (time, ydim, xdim) float64 ...
    Fpar_500m          (time, y, x) float64 ...
    _1_km_16_days_EVI  (time, ydim, xdim) float64 ...
    density            (ydim, xdim) float32 ...
    u10                (time, y, x) float64 ...
    v10                (time, y, x) float64 ...
    t2m                (time, y, x) float64 ...
    tp                 (time, y, x) float64 ...
Attributes:
    title:        MCD64A1.061 for aid0001
    Conventions:  CF-1.6
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v3.12
    references:   See README.md

In [57]:
list_xy

<xarray.Dataset>
Dimensions:      (x: 298, y: 253, time: 4629)
Coordinates:
  * x            (x) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
  * y            (y) float64 5.018e+06 5.017e+06 ... 4.786e+06 4.785e+06
  * time         (time) object 2009-12-27 00:00:00 ... 2022-08-29 00:00:00
    crs          int32 ...
    band         int32 ...
    spatial_ref  int32 ...
Data variables:
    ET_500m      (time, y, x) float64 ...
    Fpar_500m    (time, y, x) float64 ...
    density      (y, x) float32 ...
    u10          (time, y, x) float64 ...
    v10          (time, y, x) float64 ...
    t2m          (time, y, x) float64 ...
    tp           (time, y, x) float64 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [58]:
list_xdimydim

<xarray.Dataset>
Dimensions:            (ydim: 253, xdim: 298, time: 4637)
Coordinates:
  * ydim               (ydim) float64 5.018e+06 5.017e+06 ... 4.785e+06
  * xdim               (xdim) float64 3.415e+05 3.424e+05 ... 6.167e+05
  * time               (time) object 2009-12-19 00:00:00 ... 2022-08-29 00:00:00
    crs                int32 ...
Data variables:
    FireMask           (time, ydim, xdim) float64 ...
    First_Day          (time, ydim, xdim) float64 ...
    Last_Day           (time, ydim, xdim) float64 ...
    _1_km_16_days_EVI  (time, ydim, xdim) float64 ...
Attributes:
    title:        MCD64A1.061 for aid0001
    Conventions:  CF-1.6
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v3.12
    references:   See README.md
    history:      See README.md

In [51]:
# Concatenate the two datasets
cube = xr.concat([list_xy, list_xdimydim], dim='time', compat='minimal')

ValueError: 'ET_500m' is not present in all datasets.

In [35]:
# rename dimension
list_xy= list_xy.rename_dims({'x':'xdim', 'y':'ydim'})

In [59]:
cube_inner = xr.align(list_xy, list_xdimydim, join="inner")

In [11]:
cube_inner

(<xarray.Dataset>
 Dimensions:      (x: 298, y: 253, time: 4629)
 Coordinates:
   * x            (x) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
   * y            (y) float64 5.018e+06 5.017e+06 ... 4.786e+06 4.785e+06
   * time         (time) object 2009-12-27 00:00:00 ... 2022-08-29 00:00:00
     crs          int32 0
     band         int32 1
 Data variables:
     ET_500m      (time, y, x) float64 ...
     Fpar_500m    (time, y, x) float64 ...
     spatial_ref  int32 0
     density      (y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
     u10          (time, y, x) float64 ...
     v10          (time, y, x) float64 ...
     t2m          (time, y, x) float64 ...
     tp           (time, y, x) float64 ...
 Attributes:
     Conventions:  CF-1.6
     history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...,
 <xarray.Dataset>
 Dimensions:            (time: 4629, ydim: 253, xdim: 298)
 Coordinates:
   * time               (time) object 2009-12-27 0

In [61]:
cube_inner[1]

<xarray.Dataset>
Dimensions:            (time: 4629, ydim: 253, xdim: 298)
Coordinates:
  * time               (time) object 2009-12-27 00:00:00 ... 2022-08-29 00:00:00
  * ydim               (ydim) float64 5.018e+06 5.017e+06 ... 4.785e+06
  * xdim               (xdim) float64 3.415e+05 3.424e+05 ... 6.167e+05
    crs                int32 0
Data variables:
    FireMask           (time, ydim, xdim) float64 ...
    First_Day          (time, ydim, xdim) float64 ...
    Last_Day           (time, ydim, xdim) float64 ...
    _1_km_16_days_EVI  (time, ydim, xdim) float64 ...
Attributes:
    title:        MCD64A1.061 for aid0001
    Conventions:  CF-1.6
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v3.12
    references:   See README.md
    history:      See README.md

In [43]:
cube_inner[0].rename_dims({'x':'xdim', 'y':'ydim'})

<xarray.Dataset>
Dimensions:      (xdim: 298, ydim: 253, time: 4629)
Coordinates:
  * x            (xdim) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
  * y            (ydim) float64 5.018e+06 5.017e+06 ... 4.786e+06 4.785e+06
  * time         (time) object 2009-12-27 00:00:00 ... 2022-08-29 00:00:00
    crs          int32 ...
    band         int32 ...
Dimensions without coordinates: xdim, ydim
Data variables:
    ET_500m      (time, ydim, xdim) float64 ...
    Fpar_500m    (time, ydim, xdim) float64 ...
    spatial_ref  int32 ...
    density      (ydim, xdim) float32 ...
    u10          (time, ydim, xdim) float64 ...
    v10          (time, ydim, xdim) float64 ...
    t2m          (time, ydim, xdim) float64 ...
    tp           (time, ydim, xdim) float64 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

## Try by reproject_match one on the other


In [55]:
# Create the CRS
crs_sinu = rasterio.crs.CRS.from_string(
"+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs")

# Set the CRS of the data sets with hz.define_crs()
list_xy = hz.define_crs(list_xy, crs_sinu)
#list_xdimydim = hz.define_crs(list_xdimydim, crs_sinu)

RioXarrayError: Multiple grid mappings exist.

In [52]:
# Reproject_match list_xy on list_xdimydim
list_xy_reproj = list_xy.rio.reproject_match(list_xdimydim)

MissingCRS: CRS not found. Please set the CRS with 'rio.write_crs()'. Data variable: density

In [62]:
# Merge the two datasets from the cube_inner tuple
merged = xr.combine_by_coords([cube_inner[0], cube_inner[1]], combine_attrs="drop_conflicts")

In [63]:
merged

<xarray.Dataset>
Dimensions:            (x: 298, y: 253, time: 4629, ydim: 253, xdim: 298)
Coordinates:
  * x                  (x) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
  * y                  (y) float64 5.018e+06 5.017e+06 ... 4.786e+06 4.785e+06
  * time               (time) object 2009-12-27 00:00:00 ... 2022-08-29 00:00:00
    crs                int32 0
    band               int32 1
    spatial_ref        int32 0
  * ydim               (ydim) float64 5.018e+06 5.017e+06 ... 4.785e+06
  * xdim               (xdim) float64 3.415e+05 3.424e+05 ... 6.167e+05
Data variables:
    ET_500m            (time, y, x) float64 ...
    Fpar_500m          (time, y, x) float64 ...
    density            (y, x) float32 nan nan nan nan nan ... nan nan nan nan
    u10                (time, y, x) float64 ...
    v10                (time, y, x) float64 ...
    t2m                (time, y, x) float64 ...
    tp                 (time, y, x) float64 ...
    FireMask           (time, ydim, xdim) float64 ...
    First_Day          (time, ydim, xdim) float64 ...
    Last_Day           (time, ydim, xdim) float64 ...
    _1_km_16_days_EVI  (time, ydim, xdim) float64 ...
Attributes:
    Conventions:  CF-1.6
    title:        MCD64A1.061 for aid0001
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v3.12
    references:   See README.md

In [68]:
merged.reset_index('y', drop=True)

<xarray.Dataset>
Dimensions:            (x: 298, time: 4629, y: 253, ydim: 253, xdim: 298)
Coordinates:
  * x                  (x) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
  * time               (time) object 2009-12-27 00:00:00 ... 2022-08-29 00:00:00
    crs                int32 0
    band               int32 1
    spatial_ref        int32 0
  * ydim               (ydim) float64 5.018e+06 5.017e+06 ... 4.785e+06
  * xdim               (xdim) float64 3.415e+05 3.424e+05 ... 6.167e+05
Dimensions without coordinates: y
Data variables:
    ET_500m            (time, y, x) float64 ...
    Fpar_500m          (time, y, x) float64 ...
    density            (y, x) float32 nan nan nan nan nan ... nan nan nan nan
    u10                (time, y, x) float64 ...
    v10                (time, y, x) float64 ...
    t2m                (time, y, x) float64 ...
    tp                 (time, y, x) float64 ...
    FireMask           (time, ydim, xdim) float64 ...
    First_Day          (time, ydim, xdim) float64 ...
    Last_Day           (time, ydim, xdim) float64 ...
    _1_km_16_days_EVI  (time, ydim, xdim) float64 ...
Attributes:
    Conventions:  CF-1.6
    title:        MCD64A1.061 for aid0001
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v3.12
    references:   See README.md

In [43]:
# Combine the two datasets into one
final_cube = xr.combine_by_coords([list_xy, list_xdimydim], combine_attrs="override", join="left")

In [45]:
# Merge the two datasets into one
final_cube_bis = xr.merge([list_xy, list_xdimydim])

In [46]:
final_cube_bis

<xarray.Dataset>
Dimensions:            (x: 298, y: 253, time: 4637, ydim: 253, xdim: 298)
Coordinates:
  * x                  (x) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
  * y                  (y) float64 5.018e+06 5.017e+06 ... 4.786e+06 4.785e+06
  * time               (time) object 2009-12-19 00:00:00 ... 2022-08-29 00:00:00
    crs                int32 0
    band               int32 ...
  * ydim               (ydim) float64 5.018e+06 5.017e+06 ... 4.785e+06
  * xdim               (xdim) float64 3.415e+05 3.424e+05 ... 6.167e+05
Data variables:
    ET_500m            (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    Fpar_500m          (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    spatial_ref        int32 ...
    density            (y, x) float32 ...
    u10                (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    v10                (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    t2m                (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    tp                 (time, y, x) float64 nan nan nan nan ... nan nan nan nan
    FireMask           (time, ydim, xdim) float64 ...
    First_Day          (time, ydim, xdim) float64 ...
    Last_Day           (time, ydim, xdim) float64 ...
    _1_km_16_days_EVI  (time, ydim, xdim) float64 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [44]:
final_cube

<xarray.Dataset>
Dimensions:            (x: 298, y: 253, time: 4629, ydim: 253, xdim: 298)
Coordinates:
  * x                  (x) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
  * y                  (y) float64 5.018e+06 5.017e+06 ... 4.786e+06 4.785e+06
  * time               (time) object 2009-12-27 00:00:00 ... 2022-08-29 00:00:00
    crs                int32 0
    band               int32 ...
  * ydim               (ydim) float64 5.018e+06 5.017e+06 ... 4.785e+06
  * xdim               (xdim) float64 3.415e+05 3.424e+05 ... 6.167e+05
Data variables:
    ET_500m            (time, y, x) float64 ...
    Fpar_500m          (time, y, x) float64 ...
    spatial_ref        int32 ...
    density            (y, x) float32 ...
    u10                (time, y, x) float64 ...
    v10                (time, y, x) float64 ...
    t2m                (time, y, x) float64 ...
    tp                 (time, y, x) float64 ...
    FireMask           (time, ydim, xdim) float64 ...
    First_Day          (time, ydim, xdim) float64 ...
    Last_Day           (time, ydim, xdim) float64 ...
    _1_km_16_days_EVI  (time, ydim, xdim) float64 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [23]:
list_xypath_data + "list_xy.nc"

<xarray.Dataset>
Dimensions:      (x: 298, y: 253, time: 4629)
Coordinates:
  * x            (x) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
  * y            (y) float64 5.018e+06 5.017e+06 ... 4.786e+06 4.785e+06
  * time         (time) object 2009-12-27 00:00:00 ... 2022-08-29 00:00:00
    crs          int32 ...
    band         int32 ...
Data variables:
    ET_500m      (time, y, x) float64 ...
    Fpar_500m    (time, y, x) float64 ...
    spatial_ref  int32 ...
    density      (y, x) float32 ...
    u10          (time, y, x) float64 ...
    v10          (time, y, x) float64 ...
    t2m          (time, y, x) float64 ...
    tp           (time, y, x) float64 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [24]:
list_xdimydim

<xarray.Dataset>
Dimensions:            (ydim: 253, xdim: 298, time: 4637)
Coordinates:
  * ydim               (ydim) float64 5.018e+06 5.017e+06 ... 4.785e+06
  * xdim               (xdim) float64 3.415e+05 3.424e+05 ... 6.167e+05
  * time               (time) object 2009-12-19 00:00:00 ... 2022-08-29 00:00:00
    crs                int32 ...
Data variables:
    FireMask           (time, ydim, xdim) float64 ...
    First_Day          (time, ydim, xdim) float64 ...
    Last_Day           (time, ydim, xdim) float64 ...
    _1_km_16_days_EVI  (time, ydim, xdim) float64 ...
Attributes:
    title:        MCD64A1.061 for aid0001
    Conventions:  CF-1.6
    institution:  Land Processes Distributed Active Archive Center (LP DAAC)
    source:       AppEEARS v3.12
    references:   See README.md
    history:      See README.md

In [26]:
list_xy

<xarray.Dataset>
Dimensions:      (xdim: 298, ydim: 253, time: 4629)
Coordinates:
  * x            (xdim) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
  * y            (ydim) float64 5.018e+06 5.017e+06 ... 4.786e+06 4.785e+06
  * time         (time) object 2009-12-27 00:00:00 ... 2022-08-29 00:00:00
    crs          int32 0
    band         int32 1
Dimensions without coordinates: xdim, ydim
Data variables:
    ET_500m      (time, ydim, xdim) float64 ...
    Fpar_500m    (time, ydim, xdim) float64 ...
    spatial_ref  int32 0
    density      (ydim, xdim) float32 nan nan nan nan nan ... nan nan nan nan
    u10          (time, ydim, xdim) float64 ...
    v10          (time, ydim, xdim) float64 ...
    t2m          (time, ydim, xdim) float64 ...
    tp           (time, ydim, xdim) float64 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [31]:
# Merge the two datasets
final = xr.open_mfdataset( [path_data + "list_xy.nc", path_data + "list_xdimydim.nc"])

C:\Users\Quentin\anaconda3\envs\forecast\lib\site-packages\xarray\core\indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
C:\Users\Quentin\anaconda3\envs\forecast\lib\site-packages\xarray\core\indexing.py:1374: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
C:\Users

In [32]:
final

<xarray.Dataset>
Dimensions:            (x: 298, y: 253, time: 4637, ydim: 253, xdim: 298)
Coordinates:
  * x                  (x) float64 3.415e+05 3.424e+05 ... 6.157e+05 6.167e+05
  * y                  (y) float64 5.018e+06 5.017e+06 ... 4.786e+06 4.785e+06
  * time               (time) object 2009-12-19 00:00:00 ... 2022-08-29 00:00:00
    crs                int32 0
    band               int32 ...
  * ydim               (ydim) float64 5.018e+06 5.017e+06 ... 4.785e+06
  * xdim               (xdim) float64 3.415e+05 3.424e+05 ... 6.167e+05
Data variables:
    ET_500m            (time, y, x) float64 dask.array<chunksize=(4637, 253, 298), meta=np.ndarray>
    Fpar_500m          (time, y, x) float64 dask.array<chunksize=(4637, 253, 298), meta=np.ndarray>
    spatial_ref        int32 ...
    density            (y, x) float32 dask.array<chunksize=(253, 298), meta=np.ndarray>
    u10                (time, y, x) float64 dask.array<chunksize=(4637, 253, 298), meta=np.ndarray>
    v10                (time, y, x) float64 dask.array<chunksize=(4637, 253, 298), meta=np.ndarray>
    t2m                (time, y, x) float64 dask.array<chunksize=(4637, 253, 298), meta=np.ndarray>
    tp                 (time, y, x) float64 dask.array<chunksize=(4637, 253, 298), meta=np.ndarray>
    FireMask           (time, ydim, xdim) float64 dask.array<chunksize=(4637, 253, 298), meta=np.ndarray>
    First_Day          (time, ydim, xdim) float64 dask.array<chunksize=(4637, 253, 298), meta=np.ndarray>
    Last_Day           (time, ydim, xdim) float64 dask.array<chunksize=(4637, 253, 298), meta=np.ndarray>
    _1_km_16_days_EVI  (time, ydim, xdim) float64 dask.array<chunksize=(4637, 253, 298), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [9]:
list_xdimydim.time.min().values, list_xdimydim.time.max().values

(array(cftime.DatetimeJulian(2009, 12, 19, 0, 0, 0, 0, has_year_zero=False),
       dtype=object),
 array(cftime.DatetimeJulian(2022, 8, 29, 0, 0, 0, 0, has_year_zero=False),
       dtype=object))

In [10]:
# Slicing the datasets to the same time range
list_xy_sliced = list_xy.sel(time=slice('2010-02-01', '2022-01-01'))

In [11]:
list_xy.time.min().values, list_xy.time.max().values

(array(cftime.DatetimeJulian(2009, 12, 27, 0, 0, 0, 0, has_year_zero=False),
       dtype=object),
 array(cftime.DatetimeJulian(2022, 8, 29, 0, 0, 0, 0, has_year_zero=False),
       dtype=object))

In [15]:
# Convert the list_xy and list_xdimydim to gregorain calendar
list_xy= list_xy.convert_calendar('gregorian')
list_xdimydim= list_xdimydim.convert_calendar('gregorian')

In [18]:
# Print the time range of the two datasets
list_xy.time.min().values, list_xy.time.max().values

(numpy.datetime64('2009-12-27T00:00:00.000000000'),
 numpy.datetime64('2022-08-29T00:00:00.000000000'))

In [19]:
# Slicing the datasets to the same time range
list_xy_sliced = list_xy.sel(time=slice('2010-02-01', '2022-01-01'))

In [43]:
# Print the CRS of the data
print(ndvi.rio.crs)
print(era.rio.crs)

None
None


In [44]:
era

<xarray.Dataset>
Dimensions:    (longitude: 36, latitude: 21, time: 22975)
Coordinates:
  * longitude  (longitude) float32 4.0 4.1 4.2 4.3 4.4 ... 7.1 7.2 7.3 7.4 7.5
  * latitude   (latitude) float32 45.0 44.9 44.8 44.7 ... 43.3 43.2 43.1 43.0
  * time       (time) datetime64[ns] 2010-01-01T04:00:00 ... 2022-07-31T20:00:00
Data variables:
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
    t2m        (time, latitude, longitude) float32 ...
    tp         (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [45]:
# Definition of a function to define a crs for the dataray
def define_crs(dataray, crs=2154):
    """Defines a crs for the dataray"""
    return dataray.rio.write_crs(crs, inplace=True)

In [46]:
# Era datasets is in WG84 or EPSG:4326
era = define_crs(era, 4326)

CRSError: Invalid projection: epsg:4326: (Internal Proj Error: proj_create: SQLite error on SELECT name, type, coordinate_system_auth_name, coordinate_system_code, datum_auth_name, datum_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM geodetic_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name)

In [14]:
era_daily = xr.open_dataset(path_data +'era_daily.nc')

In [15]:
era_daily

<xarray.Dataset>
Dimensions:    (longitude: 36, latitude: 21, time: 4595)
Coordinates:
  * longitude  (longitude) float32 4.0 4.1 4.2 4.3 4.4 ... 7.1 7.2 7.3 7.4 7.5
  * latitude   (latitude) float32 45.0 44.9 44.8 44.7 ... 43.3 43.2 43.1 43.0
  * time       (time) datetime64[ns] 2010-01-01 2010-01-02 ... 2022-07-31
Data variables:
    u10        (time, latitude, longitude) float64 ...
    v10        (time, latitude, longitude) float64 ...
    t2m        (time, latitude, longitude) float64 ...
    tp         (time, latitude, longitude) float64 ...
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-15 13:57:35 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [17]:
print(era_daily.rio.crs)

None
